# Description

This project focuses on the critical challenge of predicting food delivery times for services such as Zomato and Swiggy, aiming to enhance operational efficiency and customer satisfaction. It involves developing a sophisticated predictive model that leverages historical data on delivery durations and incorporates the distances between pickup and delivery points. By analyzing patterns in how delivery times have varied with distance in the past, the model provides accurate delivery time estimations. This not only helps in setting realistic expectations for customers but also enables delivery services to optimize their delivery processes, reduce wait times, and improve overall service quality. The core of this project lies in its data-driven approach to solving a common yet complex problem faced by food delivery platforms, making it a valuable tool for businesses looking to streamline their delivery operations.

# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

print("All of the libraries loaded")

All of the libraries loaded


# Loading Datasets

In [12]:
path = r'D:\Data_Projects\Food_Delivery_Time_Prediction\Src\data\Delivery-time\deliverytime.txt'
data = pd.read_csv(path)

In [4]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  int64  
 3   Delivery_person_Ratings      45593 non-null  float64
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Type_of_order                45593 non-null  object 
 9   Type_of_vehicle              45593 non-null  object 
 10  Time_taken(min)              45593 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 3.8+ MB


## Columns Information

ID: order ID number 

Delivery_person_ID: ID number of the delivery partner

Delivery_person_Age: Age of the delivery partner

Delivery_person_Ratings: ratings of the delivery partner based on past deliveries

Restaurant_latitude: The latitude of the restaurant

Restaurant_longitude: The longitude of the restaurant

Delivery_location_latitude: The latitude of the delivery location

Delivery_location_longitude: The longitude of the delivery location

Type_of_order: The type of meal ordered by the customer

Type_of_vehicle: The type of vehicle delivery partner rides

Time_taken(min): The time taken by the delivery partner to complete the order

# Data Cleaning

Here, I want to manage the columns names , all of them with lower letter 

In [13]:
data.columns = data.columns.str.lower()

Checking Missing values 

In [16]:
data.isnull().sum()

id                             0
delivery_person_id             0
delivery_person_age            0
delivery_person_ratings        0
restaurant_latitude            0
restaurant_longitude           0
delivery_location_latitude     0
delivery_location_longitude    0
type_of_order                  0
type_of_vehicle                0
time_taken(min)                0
dtype: int64

Cheching Duplicated values 

In [17]:
data.duplicated().sum()

0

In this case, There are no Missing and Duplicated values , Thank god :)